## Numerical integration

In all cases we compare our methods with integrations using Gauss-Konrod quadrature from the package `QuadGK`.

In [1]:
using QuadGK
f1(x) = exp(-x^2/2)/sqrt(2*pi)
integral, err = quadgk(f1, -1.96, 1.96, rtol=1e-8)

(0.9500042097035591, 9.329403916069623e-11)

In [2]:
f2(x) = x*sin(1/x)
quadgk(f2, 0, pi)

(2.4091565809390083, 3.2917326638795794e-8)

In [16]:
f3(x) = x^4 - 3
quadgk(f3, 0, 10)

(19969.999999999996, 0.0)

### Trapezoidal rule

Add $f(x)$ evaluated at equally spaced points. Divide the values obtained by the endpoint by 2. Compute the area by adding up the function values times the trapezoid widths $h$.

In [3]:
function trapezoid(f,a,b,n)
    h = (b-a)/n
    t = range(a,b,length=n+1)
    y = f.(t)
    T = h * ( sum(y[2:n]) + 0.5*(y[1] + y[n+1]) )
    return T
end

trapezoid (generic function with 1 method)

In [17]:
trapezoid(f1, -1.96, 1.96, 1000)

0.9500039163481598

In [19]:
trapezoid(f2, eps(), pi, 100000)

2.4091565957262713

### Simpson's rule

Simpson's rule is developed in two ways in the textbook. First as a single step of Romberg extrapolation of the Trapezoidal rule. Second as a direct method using quadratic interpolation of a function instead of straight lines used in the trapezoidal rule.

In [6]:
function simpson(f, a, b, k)
    trap_table = zeros(k)
    for i in 1:k
        trap_table[i] = trapezoid(f, a, b, 2^i)[1]
    end
    simp_table = zeros(k)
    for i in 2:k
        simp_table[i] = (4*trap_table[i] - trap_table[i-1])/3 
    end
    simp_table[k]
end

simpson (generic function with 1 method)

In [7]:
simpson(f1, -1.96, 1.96, 6)  # 2^k intervals used; don't go overboard

0.9500041945590283

In [8]:
simpson(f2, eps(), pi, 8)

2.4094340233477354

[More extrapolation steps](https://en.wikipedia.org/wiki/Richardson_extrapolation) can be taken by adding extra columns to the simp_table above.

In [9]:
function simpson2(f,a,b,m)  # see exercise 5.6.4
    n = 2*m
    h = (b-a)/n
    t = range(a,b,length=n+1)
    y = f.(t)
    T = h/3 * ( sum(4 .* y[2:2:n]) + sum(2 .* y[3:2:n-1]) + (y[1] + y[n+1]) )
    return T
end

simpson2 (generic function with 1 method)

In [10]:
simpson2(f1, -1.96, 1.96, 2^5)  # 2m 

0.9500041945590284

In [11]:
simpson2(f2, eps(), pi, 2^7)

2.409434023347735

In [25]:
simpson2(x -> 1/x, eps(1.0), 10, 6) # not particularly useful

1.2509998964918078e15

In [26]:
simpson2(x -> 1/x, eps(10.0), 10, 6)

1.5637498706147884e14

## Adaptive integration

Use a method like trapezoids or Simpson. Evaluate two halves of each interval with $n$ and $2n$ nodes. Compare the two values to get an error estimate. If the error is too large, subdivde the interval and repeat the procedure.

In [28]:
function intadapt(f,a,b,tol,fa=f(a),fb=f(b),m=(a+b)/2,fm=f(m))
    # Use error estimation and recursive bisection.
    # These are the two new nodes and their f-values.
    xl = (a+m)/2;  fl = f(xl);
    xr = (m+b)/2;  fr = f(xr);
    
    # Compute the trapezoid values iteratively.
    h = (b-a)
    T = [0.,0.,0.]
    T[1] = h*(fa+fb)/2
    T[2] = T[1]/2 + (h/2)*fm
    T[3] = T[2]/2 + (h/4)*(fl+fr)
    
    S = (4T[2:3]-T[1:2]) / 3      # Simpson values
    E = (S[2]-S[1]) / 15           # error estimate
    
    if abs(E) < tol*(1+abs(S[2]))  # acceptable error?
        Q = S[2]                   # yes--done
        nodes = [a,xl,m,xr,b]      # all nodes at this level
    else
        # Error is too large--bisect and recurse.
        QL,tL = intadapt(f,a,m,tol,fa,fm,xl,fl)
        QR,tR = intadapt(f,m,b,tol,fm,fb,xr,fr)
        Q = QL + QR
        nodes = [tL;tR[2:end]]   # merge the nodes w/o duplicate
    end
    return Q,nodes
end

intadapt (generic function with 5 methods)

In [31]:
q, nodes = intadapt(f1, -1.96, 1.96, 1e-8)

(0.9500041953973487, [-1.96, -1.89875, -1.8375, -1.7762499999999999, -1.7149999999999999, -1.6843749999999997, -1.6537499999999998, -1.623125, -1.5924999999999998, -1.561875  …  1.561875, 1.5924999999999998, 1.623125, 1.6537499999999998, 1.6843749999999997, 1.7149999999999999, 1.7762499999999999, 1.8375, 1.89875, 1.96])

In [32]:
q

0.9500041953973487

In [34]:
diff(nodes)


104-element Vector{Float64}:
 0.06125000000000003
 0.06125000000000003
 0.06125000000000003
 0.06125000000000003
 0.030625000000000124
 0.030624999999999902
 0.030624999999999902
 0.030625000000000124
 0.030624999999999902
 0.030624999999999902
 0.030625000000000124
 0.030624999999999902
 0.030624999999999902
 ⋮
 0.030624999999999902
 0.030625000000000124
 0.030624999999999902
 0.030624999999999902
 0.030625000000000124
 0.030624999999999902
 0.030624999999999902
 0.030625000000000124
 0.06125000000000003
 0.06125000000000003
 0.06125000000000003
 0.06125000000000003

In [36]:
intadapt(f2, eps(), pi, 1e-8)

(2.4091451403911783, [2.220446049250313e-16, 0.001533980787885863, 0.003067961575771504, 0.004601942363657145, 0.006135923151542786, 0.006183860051164213, 0.006231796950785639, 0.006279733850407065, 0.006327670750028491, 0.006375607649649917  …  2.5034566458293668, 2.552544031041707, 2.6016314162540475, 2.650718801466388, 2.699806186678728, 2.748893571891069, 2.84706834231575, 2.945243112740431, 3.043417883165112, 3.141592653589793])

## Quadrature

Instead of using equally-spaced nodes, you can pick nodes as the roots of a family of orthogonal functions. This approach is discussed in sections 9.6, 9.7.